In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
from collections.abc import Callable
from dataclasses import dataclass
import json
from itertools import combinations_with_replacement
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import chi2
from sklearn.model_selection import KFold
import statsmodels.api as sm
from tqdm import tqdm

from data import *
from plotting import *
from regression import *
from utils import *

# Parameters

In [3]:
save = True  # Whether to save the plots

In [4]:
results_dir = 'results/hardware/13Nov/'
os.makedirs(results_dir, exist_ok=True)
os.makedirs(results_dir + 'plot_data', exist_ok=True)

In [5]:
colors = {'USA': 'blue', 'China': 'red'}


# Data preparation

In [6]:
df = pd.read_csv('data/ml_hardware_with_availability.csv')
df

,Name of the hardware,Generally available,Available in China,Available in China Notes,Manufacturer,Type,Release date,Release price (USD),FP64 (double precision) performance (FLOP/s),FP32 (single precision) performance (FLOP/s),...,Memory clock (MHz),Memory bus (bit),Tensor cores,Process size (nm),Foundry,Number of transistors (millions),Link to datasheet,Source for the price,ML models,Last modified
0,NVIDIA GB200 NVL2,N,N,NaN,NVIDIA,GPU,2024-06-02,NaN,9.000000e+13,1.800000e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.nvidia.com/en-us/data-center/gb200...,NaN,NaN,10/21/2024 9:51 PM
1,NVIDIA B200,N,N,NaN,NVIDIA,GPU,NaN,NaN,3.000000e+13,6.000000e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hyperstack.cloud/nvidia-blackwell-...,NaN,NaN,10/23/2024 9:12 PM
2,NVIDIA B100,N,N,NaN,NVIDIA,GPU,NaN,NaN,3.000000e+13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hyperstack.cloud/nvidia-blackwell-...,NaN,NaN,10/23/2024 9:12 PM
3,NVIDIA H100 SXM5 80GB,Y,N,NaN,NVIDIA,GPU,2022-09-20,"$44,489.88",3.345000e+13,6.691000e+13,...,1313.0,5120.0,528.0,5.0,TSMC,80000.0,https://resources.nvidia.com/en-us-tensor-core...,GPU was sold between 30k and 40k USD. Price is...,"Inflection-1,Inflection-2,MPT-30B (base),StarC...",11/9/2024 7:20 PM
4,Google TPU v6,Y,N,Google Cloud Platform is not available in China,Google,TPU,2024-05-14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://cloud.google.com/blog/products/compute...,NaN,NaN,10/21/2024 9:28 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,NVIDIA GeForce GTX 280,NaN,NaN,NaN,NVIDIA,GPU,2008-06-16,$649.00,7.776000e+10,6.221000e+11,...,1107.0,512.0,NaN,65.0,TSMC,1400.0,https://www.techpowerup.com/gpu-specs/geforce-...,NaN,6-layer MLP (MNIST),8/20/2024 11:38 PM
97,Huawei Ascend 910A,NaN,NaN,NaN,Huawei,GPU,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/21/2024 8:50 PM
98,Huawei Ascend 910A Pro,NaN,NaN,NaN,Huawei,GPU,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/21/2024 8:50 PM
99,Huawei Ascend 910A Premium,NaN,NaN,NaN,Huawei,GPU,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/21/2024 8:50 PM


In [7]:
df = df.dropna(subset=['Generally available', 'Available in China'])
df

,Name of the hardware,Generally available,Available in China,Available in China Notes,Manufacturer,Type,Release date,Release price (USD),FP64 (double precision) performance (FLOP/s),FP32 (single precision) performance (FLOP/s),...,Memory clock (MHz),Memory bus (bit),Tensor cores,Process size (nm),Foundry,Number of transistors (millions),Link to datasheet,Source for the price,ML models,Last modified
0,NVIDIA GB200 NVL2,N,N,NaN,NVIDIA,GPU,2024-06-02,NaN,9.000000e+13,1.800000e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.nvidia.com/en-us/data-center/gb200...,NaN,NaN,10/21/2024 9:51 PM
1,NVIDIA B200,N,N,NaN,NVIDIA,GPU,NaN,NaN,3.000000e+13,6.000000e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hyperstack.cloud/nvidia-blackwell-...,NaN,NaN,10/23/2024 9:12 PM
2,NVIDIA B100,N,N,NaN,NVIDIA,GPU,NaN,NaN,3.000000e+13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hyperstack.cloud/nvidia-blackwell-...,NaN,NaN,10/23/2024 9:12 PM
3,NVIDIA H100 SXM5 80GB,Y,N,NaN,NVIDIA,GPU,2022-09-20,"$44,489.88",3.345000e+13,6.691000e+13,...,1313.0,5120.0,528.0,5.0,TSMC,80000.0,https://resources.nvidia.com/en-us-tensor-core...,GPU was sold between 30k and 40k USD. Price is...,"Inflection-1,Inflection-2,MPT-30B (base),StarC...",11/9/2024 7:20 PM
4,Google TPU v6,Y,N,Google Cloud Platform is not available in China,Google,TPU,2024-05-14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://cloud.google.com/blog/products/compute...,NaN,NaN,10/21/2024 9:28 PM
5,NVIDIA H100 PCIe,Y,N,NaN,NVIDIA,GPU,2022-09-20,NaN,2.561000e+13,5.122000e+13,...,NaN,NaN,NaN,4.0,TSMC,80000.0,https://www.techpowerup.com/gpu-specs/h100-pci...,NaN,NaN,11/1/2024 10:44 PM
6,Google TPU v5p,Y,N,NaN,Google,TPU,2023-12-06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://cloud.google.com/tpu/docs/v5p,NaN,"Genie,Gemma 2 27B,AFM-on-device",10/17/2024 10:03 PM
7,AMD Radeon Instinct MI250X,Y,N,NaN,AMD,GPU,2021-11-08,NaN,4.790000e+13,4.790000e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.amd.com/system/files/documents/amd...,NaN,NaN,10/14/2024 9:07 PM
8,NVIDIA GeForce RTX 4090,Y,Y,NAC-eligible,NVIDIA,GPU,2022-09-20,"$1,599.00",1.290000e+12,8.260000e+13,...,1313.0,384.0,512.0,NaN,NaN,76300.0,https://www.techpowerup.com/gpu-specs/geforce-...,https://www.pcgamesn.com/nvidia/rtx-4090-relea...,"Diamond,JetFire (GPT2-LARGE)",10/21/2024 10:59 PM
9,NVIDIA A100 SXM4 80 GB,Y,N,NaN,NVIDIA,GPU,2020-11-16,NaN,9.746000e+12,1.949000e+13,...,1593.0,5120.0,432.0,7.0,TSMC,54200.0,https://www.techpowerup.com/gpu-specs/a100-sxm...,NaN,"InternLM,Llama 2-70B,Galactica,StarCoder,Lumin...",11/12/2024 2:12 AM


In [8]:
# Unify the FP16 columns
def unify_fp16(row):
    if not pd.isna(row['Tensor-FP16 performance (FLOP/s)']):
        return row['Tensor-FP16 performance (FLOP/s)']
    elif not pd.isna(row['FP16 (half precision) performance (FLOP/s)']):
        return row['FP16 (half precision) performance (FLOP/s)']
    else:
        return np.nan
df['FP16'] = df.apply(unify_fp16, axis=1)
df


/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_34299/1097342633.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FP16'] = df.apply(unify_fp16, axis=1)


,Name of the hardware,Generally available,Available in China,Available in China Notes,Manufacturer,Type,Release date,Release price (USD),FP64 (double precision) performance (FLOP/s),FP32 (single precision) performance (FLOP/s),...,Memory bus (bit),Tensor cores,Process size (nm),Foundry,Number of transistors (millions),Link to datasheet,Source for the price,ML models,Last modified,FP16
0,NVIDIA GB200 NVL2,N,N,NaN,NVIDIA,GPU,2024-06-02,NaN,9.000000e+13,1.800000e+14,...,NaN,NaN,NaN,NaN,NaN,https://www.nvidia.com/en-us/data-center/gb200...,NaN,NaN,10/21/2024 9:51 PM,1.000000e+16
1,NVIDIA B200,N,N,NaN,NVIDIA,GPU,NaN,NaN,3.000000e+13,6.000000e+13,...,NaN,NaN,NaN,NaN,NaN,https://www.hyperstack.cloud/nvidia-blackwell-...,NaN,NaN,10/23/2024 9:12 PM,2.250000e+15
2,NVIDIA B100,N,N,NaN,NVIDIA,GPU,NaN,NaN,3.000000e+13,NaN,...,NaN,NaN,NaN,NaN,NaN,https://www.hyperstack.cloud/nvidia-blackwell-...,NaN,NaN,10/23/2024 9:12 PM,1.750000e+15
3,NVIDIA H100 SXM5 80GB,Y,N,NaN,NVIDIA,GPU,2022-09-20,"$44,489.88",3.345000e+13,6.691000e+13,...,5120.0,528.0,5.0,TSMC,80000.0,https://resources.nvidia.com/en-us-tensor-core...,GPU was sold between 30k and 40k USD. Price is...,"Inflection-1,Inflection-2,MPT-30B (base),StarC...",11/9/2024 7:20 PM,9.895000e+14
4,Google TPU v6,Y,N,Google Cloud Platform is not available in China,Google,TPU,2024-05-14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://cloud.google.com/blog/products/compute...,NaN,NaN,10/21/2024 9:28 PM,9.259000e+14
5,NVIDIA H100 PCIe,Y,N,NaN,NVIDIA,GPU,2022-09-20,NaN,2.561000e+13,5.122000e+13,...,NaN,NaN,4.0,TSMC,80000.0,https://www.techpowerup.com/gpu-specs/h100-pci...,NaN,NaN,11/1/2024 10:44 PM,7.560000e+14
6,Google TPU v5p,Y,N,NaN,Google,TPU,2023-12-06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://cloud.google.com/tpu/docs/v5p,NaN,"Genie,Gemma 2 27B,AFM-on-device",10/17/2024 10:03 PM,4.590000e+14
7,AMD Radeon Instinct MI250X,Y,N,NaN,AMD,GPU,2021-11-08,NaN,4.790000e+13,4.790000e+13,...,NaN,NaN,NaN,NaN,NaN,https://www.amd.com/system/files/documents/amd...,NaN,NaN,10/14/2024 9:07 PM,3.830000e+14
8,NVIDIA GeForce RTX 4090,Y,Y,NAC-eligible,NVIDIA,GPU,2022-09-20,"$1,599.00",1.290000e+12,8.260000e+13,...,384.0,512.0,NaN,NaN,76300.0,https://www.techpowerup.com/gpu-specs/geforce-...,https://www.pcgamesn.com/nvidia/rtx-4090-relea...,"Diamond,JetFire (GPT2-LARGE)",10/21/2024 10:59 PM,3.300000e+14
9,NVIDIA A100 SXM4 80 GB,Y,N,NaN,NVIDIA,GPU,2020-11-16,NaN,9.746000e+12,1.949000e+13,...,5120.0,432.0,7.0,TSMC,54200.0,https://www.techpowerup.com/gpu-specs/a100-sxm...,NaN,"InternLM,Llama 2-70B,Galactica,StarCoder,Lumin...",11/12/2024 2:12 AM,3.120000e+14


In [9]:
ga_df = df[df['Generally available'] == 'Y']
china_df = df[df['Available in China'] == 'Y']


In [10]:
ga_nvidia_df = ga_df[ga_df['Manufacturer'] == 'NVIDIA']
ga_other_df = ga_df[ga_df['Manufacturer'] != 'NVIDIA']
china_nvidia_df = china_df[china_df['Manufacturer'] == 'NVIDIA']
china_other_df = china_df[china_df['Manufacturer'] != 'NVIDIA']

In [11]:
# Plot FP16 performance over time in ga_df and china_df
# Use graph_objects to plot two lines on the same plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ga_nvidia_df['Release date'],
    y=ga_nvidia_df['FP16'],
    mode='markers',
    text=ga_nvidia_df['Name of the hardware'],
    name='NVIDIA, Generally available',
    line=dict(color=colors['USA'])
))
fig.add_trace(go.Scatter(
    x=china_nvidia_df['Release date'],
    y=china_nvidia_df['FP16'],
    mode='markers',
    text=china_nvidia_df['Name of the hardware'],
    name='NVIDIA, Available in China',
    line=dict(color=colors['China'])
))

fig.add_trace(go.Scatter(
    x=ga_other_df['Release date'],
    y=ga_other_df['FP16'],
    mode='markers',
    text=ga_other_df['Name of the hardware'],
    name='Other, Generally available',
    marker=dict(color=colors['USA'], symbol='x')
))
fig.add_trace(go.Scatter(
    x=china_other_df['Release date'],
    y=china_other_df['FP16'],
    mode='markers',
    text=china_other_df['Name of the hardware'],
    name='Other, Available in China',
    marker=dict(color=colors['China'], symbol='x')
))

annotations = ['NVIDIA H100 SXM5 80GB', 'NVIDIA HGX H20', 'Huawei Ascend 910B']
for annotation in annotations:
    fig.add_annotation(
        x=df.loc[df['Name of the hardware'] == annotation, 'Release date'].values[0],
        y=np.log10(df.loc[df['Name of the hardware'] == annotation, 'FP16'].values[0]),
        text=annotation,
        showarrow=True,
        ax=0,
        ay=-20
    )

fig.update_yaxes(type='log')
fig.update_layout(
    template='plotly_white',
    title='Maximum FP16 performance over time',
    xaxis_title='Release date',
    yaxis_title='FP16 performance (FLOP/s)',
    legend_title='Availability',
)

# Save plot
save_plot(fig, results_dir, 'fp16_performance_over_time')

fig.show()

In [13]:
col = 'Memory bandwidth (byte/s)'
# Plot FP16 performance over time in ga_df and china_df
# Use graph_objects to plot two lines on the same plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ga_nvidia_df['Release date'],
    y=ga_nvidia_df['Memory bandwidth (byte/s)'],
    mode='markers',
    text=ga_nvidia_df['Name of the hardware'],
    name='NVIDIA, Generally available',
    line=dict(color=colors['USA'])
))
fig.add_trace(go.Scatter(
    x=china_nvidia_df['Release date'],
    y=china_nvidia_df[col],
    mode='markers',
    text=china_nvidia_df['Name of the hardware'],
    name='NVIDIA, Available in China',
    line=dict(color=colors['China'])
))

fig.add_trace(go.Scatter(
    x=ga_other_df['Release date'],
    y=ga_other_df[col],
    mode='markers',
    text=ga_other_df['Name of the hardware'],
    name='Other, Generally available',
    marker=dict(color=colors['USA'], symbol='x')
))
fig.add_trace(go.Scatter(
    x=china_other_df['Release date'],
    y=china_other_df[col],
    mode='markers',
    text=china_other_df['Name of the hardware'],
    name='Other, Available in China',
    marker=dict(color=colors['China'], symbol='x')
))

fig.update_yaxes(type='log')
# Annotate NVIDIA H100 SXM5 80GB
annotations = ['NVIDIA H100 SXM5 80GB', 'NVIDIA HGX H20', 'Huawei Ascend 910B', 'AMD Radeon Instinct MI300']
for annotation in annotations:
    fig.add_annotation(
        x=df.loc[df['Name of the hardware'] == annotation, 'Release date'].values[0],
        y=np.log10(df.loc[df['Name of the hardware'] == annotation, col].values[0]),
        text=annotation,
        showarrow=True,
        ax=0,
        ay=-20
    )

fig.update_layout(
    template='plotly_white',
    title='Memory bandwidth over time',
    xaxis_title='Release date',
    yaxis_title='Memory bandwidth (byte/s)',
    legend_title='Availability',
)

# Save plot
save_plot(fig, results_dir, 'memory_bandwidth_over_time')

fig.show()